# Классификация новостей по тегам на основе текстового содержимого

**Цель:** Решение задачи многозначной многоклассовой классификации новостей на основе ключевых слов из текстового описания.

## Сбор данных. Парсинг сайта

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import random
import time
import requests
from tqdm import tqdm
from datetime import datetime
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

## Сбор данных

В качестве источника данных будет использован [`медиапортал`](https://media.ssmu.ru/news/) Сибирского государственного медицинского университета

In [2]:
target_page = "https://media.ssmu.ru/news/"

Страница использует **ленивую подгрузку контента** (`lazy loading`) через **AJAX**. Изначально загружается ограниченное количество новостей, а каждые следующие 10 подгружаются при нажатии на кнопку `Показать ещё 10`, которая отправляет **асинхронный HTTP-запрос** к серверу.

### Парсинг метаданных новостей (без текста, включая ссылки)

Получим **ссылки на страницы** всех новостей, параллельно извлекая **дату публикации**, **заголовок**, **просмотры** и **теги**.<br />Далее используем ссылки для парсинга **полных текстовых описаний**.

In [3]:
driver = webdriver.Chrome()
driver.get(target_page)

start_time = time.time()

# Загрузка карточек всех новостей на страницу
while True:
    try:
        # Ждём появления и кликабельности кнопки
        show_more = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "js-show-more"))
        )
        
        # Прокручиваем страницу до кнопки (имитация поведения пользователя)
        driver.execute_script("arguments[0].scrollIntoView(true);", show_more)

        show_more.click()  # Нажимаем на кнопку

        time.sleep(random.uniform(1, 3))  # Случайная задержка
        
    except TimeoutException:
        print("Кнопка `Показать ещё 10` больше не найдена. Завершаем цикл.")
        break 

    except StaleElementReferenceException:
        print("Элемент изменился после нахождения. Повторим попытку.")
        continue
        
exec_time = time.time() - start_time  # Время выполнения в секундах
print(f"\nВремя выполнения: {exec_time:.2f} секунд ({exec_time / 60:.2f} минут).")

Кнопка `Показать ещё 10` больше не найдена. Завершаем цикл.

Время выполнения: 427.77 секунд (7.13 минут).


In [4]:
months = {
    "января": "January", "февраля": "February", "марта": "March", "апреля": "April",
    "мая": "May", "июня": "June", "июля": "July", "августа": "August",
    "сентября": "September", "октября": "October", "ноября": "November", "декабря": "December"
}

news = []  # Список для хранения новостей
all_news = driver.find_elements(By.CSS_SELECTOR, "article.news")  # Получение карточек новостей
seen_links = set()  # Множество для проверки дубликатов


# Извлечение метаинформации новостей
for new in tqdm(all_news, desc="Парсинг новостей", unit="новость"):
    title = new.find_element(By.CSS_SELECTOR, ".news__title > a")
    link = title.get_attribute("href")
    
    if link in seen_links:
        continue
        
    seen_links.add(link)
    
    tags = new.find_elements(By.CSS_SELECTOR, ".news__tag.tag")
    views = new.find_element(By.CSS_SELECTOR, ".news__views > span")
    publication_date = new.find_element(By.CSS_SELECTOR, ".news__date")
    
    day, month, year = publication_date.text.strip().split()

    news.append({"Published": datetime.strptime(f"{day} {months[month]} {year}", "%d %B %Y").date(), 
                 "Title": title.text, "Link": link, "Views": views.text,
                 "Tags": ", ".join([tag.text.strip() for tag in tags])})

    
print(f"Всего собрано: {len(news)}")                 
print("\nСрез из 10 новостей:")
for new in news[100:110]:
    print(f"{new['Title']} -> {new['Link']}\n")
    
# Закрытие браузера после завершения работы    
driver.quit()

Парсинг новостей: 100%|███████████████████████████████████████████████████████| 1902/1902 [02:07<00:00, 14.91новость/s]


Всего собрано: 1902

Срез из 10 новостей:
Эксперт психологического центра СибГМУ рассказала, как влиться в рабочие будни после январских праздников -> https://media.ssmu.ru/news/ekspert-psikhologicheskogo-tsentra-sibgmu-rasskazala-kak-vlitsya-v-rabochie-budni-posle-yanvarskikh-/

Апельсины: в чем польза, есть ли вред, как выбирать — рассказывает эксперт СибГМУ -> https://media.ssmu.ru/news/apelsiny-v-chem-polza-est-li-vred-kak-vybirat-rasskazyvaet-ekspert-sibgmu/

Юбилей празднует профессор кафедры анестезиологии, реаниматологии и интенсивной терапии СибГМУ Виталий Шипаков -> https://media.ssmu.ru/news/yubiley-prazdnuet-professor-kafedry-anesteziologii-reanimatologii-i-intensivnoy-terapii-sibgmu-vital/

Юбилей празднует старший преподаватель кафедры иностранных языков Наталья Стасюк -> https://media.ssmu.ru/news/yubiley-prazdnuet-starshiy-prepodavatel-kafedry-inostrannykh-yazykov-natalya-stasyuk/

Поздравление Министра здравоохранения Российской Федерации Михаила Мурашко с Новым годом 

### Парсинг контента новостей

In [5]:
ua = UserAgent()  # Объект для генерации случайных User-Agent

for new in tqdm(news, desc="Парсинг контента новостей", unit="страница"):
    time.sleep(random.uniform(0.1, 0.6))  # Задержка для избежания блокировок
    
    headers = {"User-Agent": ua.random}  # Генерация случайного User-Agent
    response = requests.get(new["Link"], headers=headers)
    
    if response.ok:
        page = response.content
        dom = BeautifulSoup(page, "html.parser")
        
        # Извлечение текстового содержимого новости
        text_content = dom.select(".text-content")
        text = " ".join([text_block.text.strip() for text_block in text_content])
        
        new["Content"] = text  # Добавление контента в соответствующий словарь

Парсинг контента новостей: 100%|█████████████████████████████████████████████| 1902/1902 [34:23<00:00,  1.08s/страница]


Проверка ссылок показала, что в этих новостях действительно нет текстового описания, вместо него видеоролики

In [6]:
removed_news = [(new["Title"], new["Link"])
                for new in news 
                if not new.get("Content")]  # Новости без текстового контента

for title, link in removed_news:
    print(f"{title} -> {link}\n")
    
print(f"Всего потенциальных новостей для удаления из датасета: {len(removed_news)}")

news = [new for new in news if new.get("Content")]  # Оставляем только новости с контентом

Поздравление от министра здравоохранения Российской Федерации Михаила Мурашко. -> https://media.ssmu.ru/news/pozdravleniya-ot-ministra-zdravookhraneniya-rossiyskoy-federatsii-mikhaila-murashko/

Поздравление Министра здравоохранения Российской Федерации Михаила Мурашко с Днем российского студенчества -> https://media.ssmu.ru/news/pozdravlenie-ministra-zdravookhraneniya-rossiyskoy-federatsii-mikhaila-murashko-s-dnem-rossiyskogo-s2025/

Поздравление Министра здравоохранения Российской Федерации Михаила Мурашко с Новым годом -> https://media.ssmu.ru/news/pozdravlenie-ministra-zdravookhraneniya-rossiyskoy-federatsii-mikhaila-murashko-s-novym-godom2025/

Всего потенциальных новостей для удаления из датасета: 3


### Преобразование в DataFrame и сохранение в файл

In [7]:
data = pd.DataFrame(news)
data.to_csv("news_data.csv", index=False, encoding="utf-8")  # Сохранение в CSV
data

,Published,Title,Link,Views,Tags,Content
0,2025-03-12,СибГМУ выступил соорганизатором международной ...,https://media.ssmu.ru/news/sibgmu-vystupil-soo...,69,Наука,СибГМУ уже второй раз принял участие в междуна...
1,2025-03-11,В СибГМУ подведены итоги отбора участников про...,https://media.ssmu.ru/news/v-sibgmu-zavershils...,146,Образование,Проект был запущен в 2022 году в рамках програ...
2,2025-03-10,"Сотрудник СибГМУ рассказала, как сохранить здо...",https://media.ssmu.ru/news/sotrudnik-sibgmu-ra...,223,Наука,Ассистент кафедры анатомии человека с курсом т...
3,2025-03-08,Поздравление ректора СибГМУ с Международным же...,https://media.ssmu.ru/news/pozdravlenie-rektor...,343,Университет,"Дорогие женщины, искренне поздравляю вас с Меж..."
4,2025-03-07,Юбилей празднует специалист по учебно-методиче...,https://media.ssmu.ru/news/yubiley-prazdnuet-s...,377,Университет,"Сегодня, 7 марта, день рождения отмечает специ..."
...,...,...,...,...,...,...
1894,2021-05-28,«Человек года в СибГМУ»: студент Владислав Бел...,https://media.ssmu.ru/news/chelovek-goda-v-sib...,72,"Университет, Мир СибГМУ",Одним из победителей конкурса «Человек года-20...
1895,2021-04-27,«Человек года в СибГМУ»: студентка Анастасия Р...,https://media.ssmu.ru/news/chelovek-goda-v-sib...,74,"Университет, Студенчество",Одним из победителей конкурса «Человек года-20...
1896,2021-04-13,«Человек года в СибГМУ»: Алена Шадрина о возмо...,https://media.ssmu.ru/news/chelovek-goda-v-sib...,130,"Университет, Студенчество",Одним из победителей конкурса «Человек года-20...
1897,2021-03-26,«Человек года в СибГМУ»: Екатерина Трифонова о...,https://media.ssmu.ru/news/chelovek-goda-v-sib...,86,"Университет, Студенчество",Одним из победителей конкурса «Человек года-20...
